# TensorBoard的用法

## 简介
　　前面我们说过了利用TensorFlow进行模型搭建的基本方法，这一节主要是在上一节的内容上进一步研究模型的可视化问题。模型可视化主要是用TensorBoard进行的，这里给出应用实例，实例内容来自[Visualizing Learning](https://www.tensorflow.org/get_started/summaries_and_tensorboard),是基于一个神经网络进行的,而不是基于cnn的,而且对一些内容进行了更正。在此基础上还融合了[Embedding Visualization](https://www.tensorflow.org/get_started/embedding_viz) 中的代码。并且会在我的blog中进行更详细的分析。

<font color= BlueViolet size=5>程序全部是基于win10的</font>  
$\color{BlueViolet}{程序全部是基于win10的}$

**---**

### 这里首先有几个会用到的概念：  
　　1.**命名空间**：为了能够规范化的显示每个节点的名称，如果设置了命名空间则这个空间下的名字都会有一个相同的起始名，如：在
with tf.name_scope('input'):下的所有节点都会自动被命名为input/xxx的格式。同一个命名空间的数据在TensorBoard中会放在一起  
　　2.**logdir日志路径**：这里面存放了所有的汇总数据。  
　　3.**Embedding**: 可以看成一种映射，PCA和TSNE都是其中的一种方法
***
### 这里还有几个常用的函数：
1. [tf.summary系列](https://www.tensorflow.org/api_guides/python/summary),主要是两类：Summary Ops（汇总Operation）和writing Summaries（写入汇总的方法)
 1. 其中Summary Ops主要分两类就是tf.summary.scalar/histogram/image/audio这些生成summary的方法，还有tf.summary.merge/merge_all的汇总全部summary的方法。
 2. writing Summaries只要记住有tf.summary.FileWriter这个写入类，其初始化参数是(log_dir，graph=None)即可（还有其他几个参数就不写了因为这里用不到）,他有一个add_summary(summary,global_step=None)可以同时记录迭代数和summary；还有一个add_run_metadata(run_metadata,tag)方法可以记录元数据（tag就是元数据的名称）  
2. [tf.RunMetadata](https://www.tensorflow.org/api_docs/python/tf/RunMetadata)  
   这个是定义一个元数据，sess.run方法和tf.summary.add_run_metadata都会用到，可以记录占用内存等信息 
3. [tf.RunOptions](https://www.tensorflow.org/api_docs/python/tf/RunOptions)  
   和上面的类似，是run方法的一个需要的参数，是TensorFlow的运行选项。

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
#定义所需全局量：常量，如学习率，迭代次数
max_step = 1000
learning_rate = 0.001
drop_out = 0.5
log_dir = 'D:/spring1617_assignment2/assignment2/tmp'

In [3]:
#为了简便首先把所需要的都用函数封装起来
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#我们希望可视化变量的内容
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)#把mean放入scalar汇总中
        with tf.name_scope('stddev'):
            #这里不知道为什么要多加一个命名空间
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))#计算标准差
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))#最大值
        tf.summary.scalar('min', tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram', var)#分布图，？？？？？？二维的是怎么算的
        
#定义一个建立一个神经网络，并加入汇总的函数,否则层数很多就会很麻烦
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        weights = weight_var([input_dim, output_dim])#这里是个二维的
        variable_summaries(weights)#进行汇总
    with tf.name_scope('biases'):
        biases = bias_var([output_dim])
        variable_summaries(biases)
    with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate) #这里是要分布直方图，
    activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)#可以和前面进行前后对比
    return activations

#定义一个feed_dict函数方便区分训练和测试
def feed_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(100)
        k = drop_out
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x: xs, y_: ys, prob: k}

上面所需的函数已经都封装好了，下面就是开始设计图的过程了,建立会话开始搭建图

In [4]:
sess = tf.InteractiveSession()
#设计输入
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
#这个完全是为了汇总为image后进行可视化    
with tf.name_scope('input_reshape'):
    image_shaped = tf.reshape(x,[-1, 28, 28, 1])
    tf.summary.image('input', image_shaped)

#首先搭建一个nn,得到一个500维的输出
hidden1 = nn_layer(x, 784, 500, 'layer1')
#经过一个drop_out
with tf.name_scope('dropout'):
    prob = tf.placeholder(tf.float32)
    tf.summary.scalar('drop_out_keep_prob', prob)
    dropped = tf.nn.dropout(hidden1, prob)
    
#搭建第二个layer得到输出
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)#tf.identity就是一个恒等函数

with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

<tf.Tensor 'cross_entropy_1:0' shape=() dtype=string>

接下去就是优化和训练及评估过程了

In [5]:
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_pre = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_pre, tf.float32))
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

直接获取所有的汇总操作，接着在训练之前设置好summary的写入节点存放训练的日志数据，而且**在训练的sess.graph加入训练过程的记录仪，这样在TensorBoard的$\color{DarkTurquoise}{GRAPH}$窗口就能展示整个计算图的可视化效果** 

In [6]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')

#初始化训练图
tf.global_variables_initializer().run()

## 这里开始对上面的图进行训练
* 毎10次进行1次对测试数据的测试，并输出结果把结果和迭代次数写到测试对应的日志数据中
* 每100次把TensorFlow的元信息，包括：运算时间和内存占用等记录下来写入训练日志中(可选)
tf.RunOpetions定义了一些运行选项是一个proto，这个也是在一本书上看到的，有精力的可以详细看一看这个是怎么写的
同理RunMetadata也是proto，这两个在sess.run中都是可选参数。

In [7]:
saver = tf.train.Saver()#这个用于创建模型的保存器，只有有保存器才能够进行Embedding
for i in range(max_step):
    if i%10 == 0:
        summary, acc = sess.run([merged, accuracy], feed_dict = feed_dict(False))
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    elif i%100 == 99:
        run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()#产生元数据
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True),options = run_options, run_metadata=run_metadata)
        #开始写入结果
        train_writer.add_summary(summary, i) 
        train_writer.add_run_metadata(run_metadata, 'step%03d'%i)
        saver.save(sess,log_dir+'/model.ckpt')
        print('Adding run metada for',i)
    else:
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
#关闭writer
train_writer.close()
test_writer.close()

Accuracy at step 0: 0.0845
Accuracy at step 10: 0.6867
Accuracy at step 20: 0.7982
Accuracy at step 30: 0.8485
Accuracy at step 40: 0.875
Accuracy at step 50: 0.883
Accuracy at step 60: 0.8958
Accuracy at step 70: 0.9019
Accuracy at step 80: 0.9076
Accuracy at step 90: 0.9107
Adding run metada for 99
Accuracy at step 100: 0.9126
Accuracy at step 110: 0.9158
Accuracy at step 120: 0.9153
Accuracy at step 130: 0.9155
Accuracy at step 140: 0.9201
Accuracy at step 150: 0.924
Accuracy at step 160: 0.9245
Accuracy at step 170: 0.927
Accuracy at step 180: 0.9301
Accuracy at step 190: 0.9321
Adding run metada for 199
Accuracy at step 200: 0.9324
Accuracy at step 210: 0.9322
Accuracy at step 220: 0.9311
Accuracy at step 230: 0.9332
Accuracy at step 240: 0.9334
Accuracy at step 250: 0.937
Accuracy at step 260: 0.9347
Accuracy at step 270: 0.9374
Accuracy at step 280: 0.9396
Accuracy at step 290: 0.9411
Adding run metada for 299
Accuracy at step 300: 0.9432
Accuracy at step 310: 0.9425
Accuracy at

此时数据已经写入了logdir中